In [128]:
import pandas as pd
import numpy as np
import random
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.metrics
import sklearn.metrics.pairwise
import math

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
movies = pd.read_csv('./ml-latest-small/movies.csv')
tags = pd.read_csv('./ml-latest-small/tags.csv')

#Split 80%/20% between train and test
train_ratings, test_ratings = sklearn.model_selection.train_test_split(ratings, test_size=0.2)

ImportError: No module named suprise

Content-based Filtering
============
Content-based filtering methods are based on a description of the item and a profile of the user’s preferences

Uses user and item features to create a recommendation model. Think of it as like any regression or classification problem in machine learning

Example: Linear Regression
------

We'll use the movie genres as the movie features. Process it as one hot encoding vector for each movie

In [46]:
#Process movie features
genres = set()
movie_genres = {}
for index, row in movies.iterrows():
    genres.add(row['genres'])
    movie_genres[row['movieId']] = row['genres'].split("|")
    
genres_a = []
for g in genres:
    genres_a.append(g)
    
movie_features = {}
for key in movie_genres:
    val = movie_genres[key]
    movie_features[key] = []
    for g in genres_a:
        movie_features[key].append(1 if g in val else 0)

In [58]:
lr = sklearn.linear_model.LinearRegression()

train_features = []
train_labels = []

test_features = []
test_labels = []

for index, row in train_ratings.iterrows():
    train_features.append(movie_features[row['movieId']])
    train_labels.append(row['rating'])
    
for index, row in test_ratings.iterrows():
    test_features.append(movie_features[row['movieId']])
    test_labels.append(row['rating'])
    
lr.fit(train_features, train_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Next step is to evaluate our models. A common metric for evaluating recommendation algorithms is the Root Mean Squared Error (RMSE). 

$$RMSE = \sqrt{\frac{1}{N} \sum\limits_{i}^{N} (y_i-\hat{y}_i)^2}$$

In [67]:
predicted_ratings = lr.predict(test_features)

rmse = math.sqrt(sklearn.metrics.mean_squared_error(predicted_ratings, test_labels))
print "RMSE:", rmse

RMSE: 1.03700622971


An un-regularized linear regression model with just using movie genres is getting an RMSE 1.037! That's not too bad already. You can probably even continue to try to improve this model by incorporating some user features (not available in movielens 100k) or adding regularization to the regression

Collaborative Filtering
============

In collaborative filtering, we can make recommendations using only information about the past preferences of the the user along with other users. We don't need any user or item features, and it generally works better than content-based recommendation.

Visualization:
------
<img src="./images/Collaborative_filtering-14.tiff" style="height:300px;width:300px"/>

<img src="./images/Collaborative_filtering-16.tiff" style="height:300px;width:300px"/>

K-Nearest Neighbors
--------------------

A common form of collaborative filtering is the nearest neighbors method. The intuition behind it is:
1. Get the similarities between users or items (More on this later)
2. Weigh the recommendations of the nearest neighbors of the user based on the similarities

In [101]:
#Get lookup dict or ratings from movie to user perspective
movie_users = {}
user_movies = {}
for index, row in train_ratings.iterrows():
    if row['movieId'] not in movie_users:
        movie_users[row['movieId']] = {}
        
    if row['userId'] not in user_movies:
        user_movies[row['userId']] = {}
        
    movie_users[row['movieId']][row['userId']] = row['rating']
    user_movies[row['userId']][row['movieId']] = row['rating']
     

We need a similarity metric. A simple one we can use is the cosine similarity between two vectors A and B. In this setting A and B are the ratings each user has given to movies that they have <b>both watched</b>.

$$\cos(A,B)=\frac{\sum\limits_{i}^{N}A_iB_i}{\sqrt{\sum\limits_{i}^{N}A_i^2} \sqrt{\sum\limits_{i}^{N}B_i^2}} $$

In [200]:
#Calculate similarity ratings for all user combinations
#Uses cosine similarity
similarities = {}

for user1 in users:
    if user1 not in similarities:
        similarities[user1] = {}
    
    for user2 in users:
        if user2 in similarities[user1] or user1 == user2:
            continue
        
        user1_ratings = []
        user2_ratings = []
        
        for key in user_movies[user1]:
            watched_users = movie_users[key]
            if user1 in watched_users and user2 in watched_users:
                user1_ratings.append(watched_users[user1])
                user2_ratings.append(watched_users[user2])
                
        if len(user1_ratings) > 0:
            if user2 not in similarities:
                similarities[user2] = {}
            
            sim = np.dot(user1_ratings, user2_ratings) / (np.linalg.norm(user1_ratings) * np.linalg.norm(user2_ratings))
            similarities[user1][user2] = sim
            similarities[user2][user1] = sim
            

Using the similarities, we can use nearest neighbor regression using the following formula: 

$$R_{x,y} = \frac{\sum\limits_{z} Sim_{x,z}R_{z,y}}{\sum\limits_{z} Sim_{x,z}}$$

In [172]:
#Nearest neighbor regression
def knn_predict(k, userId, movieId):
    watched_users = movie_users[movieId]
    
    sim_scores = []
    rating_scores = []
    
    for w in watched_users:
        if w in similarities[userId]:
            sim_scores.append(similarities[userId][w])
            rating_scores.append(watched_users[w])
    
    #Got lazy. Used pandas DataFrames to sort by similarity and get the nearest neighbors
    if len(sim_scores) > 0:
        sim_df = pd.DataFrame(data={'sim' : sim_scores, 'rating' : rating_scores})
        sorted_sim_df = sim_df.sort_values(by=['sim'], ascending = False)
        knearest = sorted_sim_df.head(k)
        
        numerator = 0
        denomenator = 0
        for index, row in knearest.iterrows():
            #print row['sim'], row['rating']
            numerator += row['sim'] * row['rating']
            denomenator += row['sim']
        #print "Quotient", numerator, denomenator, numerator / denomenator  
        return numerator / denomenator
    else: #What to do if user has no similarities??
        return 2.5
    
    


In [180]:
predicted_ratings = []
actual_ratings = []
K = 5
for index, row in test_ratings.iterrows():
    if row['movieId'] not in movie_users:
        continue
        
    pred = knn_predict(50, row['userId'], row['movieId'])
    predicted_ratings.append(pred)
    actual_ratings.append(row['rating'])
    
rmse = math.sqrt(sklearn.metrics.mean_squared_error(predicted_ratings, actual_ratings))
print "RMSE:", rmse

No sim
No sim
RMSE: 0.985502162215


RMSE of 0.98! Already better than the linear regression example, but hopefully can still be improved. Two main things you can fiddle with here aside from K:

1. Use item similarity instead of user. This was pioneered by Amazon in 1998 and documented in papers they published in 2001 and 2003 (https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf). This works especially better when there are fewer items than users, so similarities will be richer due to more intersections.

2. Try different similarity metrics. For example: Pearson correlation which goes from -1 to 1 instead of just 0 to 1.

Matrix Factorization
--------

A more powerful form of collaborative filtering is matrix factorization. The basic intuition is that it models the user-item ratings/preferences as an $M \times N$ matrix $R$ where $M$ is the number of users and $N$ is the number of items.
<table>
    <tr>
        <th></th>
        <th>Movie1</th>
        <th>Movie2</th>
        <th>Movie3</th>
        <th>Movie4</th>
    </tr>
    <tr>
        <th>User1</th>
        <td>5</td>
        <td></td>
        <td>1.5</td>
        <td>3.5</td>
    </tr>
    <tr>
        <th>User2</th>
        <td>1</td>
        <td>3.5</td>
        <td></td>
        <td>4</td>
    </tr>
    <tr>
        <th>User3</th>
        <td></td>
        <td>2</td>
        <td>4</td>
        <td>5</td>
    </tr>
</table>

There are different kinds of matrix factorization methods for CF. A simple one is Probabilistic Matrix Factorization (https://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf). In PMF the ratings matrix $R$ is decomposed into an $M \times K$ matrix $U$ and a $N \times K$ matrix $V$ such that

$$ R = U^TV$$

### Learning $U$ and $V$ ######

So how do we learn U and V? We can do gradient descent using Alternating Least Squares to minimize the following error function:

$$ 
E = \frac{1}{2} \sum\limits_{i}^{N}\sum\limits_{j}^{M}(R_{i,j} - U_{i}^{T}V_{j})^2 
+ \frac{\lambda_U}{2} \sum\limits_{i}^{N} \|{U_i}\|_{Fro}^2
+ \frac{\lambda_V}{2} \sum\limits_{j}^{M} \|{V_j}\|_{Fro}^2
$$

In [ ]:
def get_error(user_matrix, item_matrix, actual, lu, li):
    error = 0
    for u in actual:
        for i in actual[u]:
            pred = np.dot(user_matrix[u], item_matrix[i])
            error += (pred - actual[u][i]) ** 2
            
    error /= 2
    
    u_norm = 0
    for u in user_matrix:
        for val in user_matrix[u]:
            u_norm += val ** 2
    i_norm = 0
    for i in item_matrix:
        for val in item_matrix[i]:
            i_norm += val ** 2
            
    error += lu * u_norm / 2
    error += li * i_norm / 2
              
    return error
    

In [141]:
#Derivative with respect to U
def dE_u(user_matrix, item_matrix, actual, lu, u, index):
    derivative = 0
    for i in actual[u]:
        pred = np.dot(user_matrix[u], item_matrix[i])
        derivative += (pred - actual[u][i]) * item_matrix[i][index]
        
    derivative += lu * user_matrix[u][index]
    
    return derivative

In [146]:
#Derivative with respect to V
def dE_i(user_matrix, item_matrix, actual, li, i, index):
    derivative = 0
    watched_users = movie_users[i]
    for u in watched_users:
        pred = np.dot(user_matrix[u], item_matrix[i])
        derivative += (pred - actual[u][i]) * user_matrix[u][index]
        
    derivative += li * item_matrix[i][index]
    
    return derivative

In Alternating Least Squares, we get the gradients of $E$ by fixing $V$ as constants and getting the derivative of $E$ with respect to $U$ and vice versa. We alternate between both gradients when we update the matrices.

In [192]:
#Alternating Least Squares
def learn(user_matrix, item_matrix, actual, lu, li, learning_rate):
    counter = 0
    while counter < 1000:
        error = get_error(user_matrix, item_matrix, actual, lu, li)
        print "Counter:", counter, "Error:", error #Error should trend downwards over time
        
        #Treat V as contant, update U with its gradients
        if counter % 2 == 0:
            user_derivatives = {}      
            for u in user_matrix:
                user_derivatives[u] = []
            
                for index in range(len(user_matrix[u])):
                    user_derivatives[u].append(dE_u(user_matrix, item_matrix, actual, lu, u, index))
                    
            for u in user_matrix:
                for index in range(len(user_matrix[u])):
                    user_matrix[u][index] -= learning_rate * user_derivatives[u][index]

        #Treat U as constant, update V with its gradients
        else:
            item_derivatives = {}
            for i in item_matrix:
                item_derivatives[i] = []
                for index in range(len(item_matrix[i])):
                    item_derivatives[i].append(dE_i(user_matrix, item_matrix, actual, li, i, index))
                    
            for i in item_matrix:
                for index in range(len(item_matrix[i])):
                    item_matrix[i][index] -= learning_rate * item_derivatives[i][index]
        
        counter += 1
        
    return (user_matrix, item_matrix)

In [ ]:
user_matrix = {}
item_matrix = {}
learning_rate = 0.0001
lu = 0.1
li = 0.1
K = 10

for u in user_movies:
    user_matrix[u] = random.sample(range(1, 100), K)
    for index in range(K):
        user_matrix[u][index] /= float(100)
    
for i in movie_users:
    item_matrix[i] = random.sample(range(1, 100), K)
    for index in range(K):
        item_matrix[i][index] /= float(100)
    
learned_users, learned_items = learn(user_matrix, item_matrix, user_movies, lu, li, learning_rate)

Counter: 0 Error: 106447.02087946555
Counter: 1 Error: 99701.0194983682
Counter: 2 Error: 97984.660477298
Counter: 3 Error: 93146.66490353506
Counter: 4 Error: 91571.27329967637
Counter: 5 Error: 87822.56977110129
Counter: 6 Error: 86381.5704499992
Counter: 7 Error: 83340.10850548765
Counter: 8 Error: 82024.67166494431
Counter: 9 Error: 79487.52627964206
Counter: 10 Error: 78287.96993193732
Counter: 11 Error: 76133.73350290785
Counter: 12 Error: 75040.20295635345
Counter: 13 Error: 73188.74283798868
Counter: 14 Error: 72191.58788909971
Counter: 15 Error: 70585.9569403898
Counter: 16 Error: 69675.9574113858
Counter: 17 Error: 68273.51134701163
Counter: 18 Error: 67442.0122941872
Counter: 19 Error: 66209.64304255143
Counter: 20 Error: 65448.629619725376
Counter: 21 Error: 64359.98058261872
Counter: 22 Error: 63662.11040591029
Counter: 23 Error: 62695.82153993509
Counter: 24 Error: 62054.4268553469
Counter: 25 Error: 61192.95702823595
Counter: 26 Error: 60602.025836676854
Counter: 27 Erro

In [209]:
predicted_ratings = []
actuals = []
not_count = 0
for index, row in test_ratings.iterrows():
    if row['movieId'] not in movie_users:
        continue
        
    pred = np.dot(learned_users[row['userId']], learned_items[row['movieId']])
    predicted_ratings.append(pred)
    actuals.append(row['rating'])

rmse = math.sqrt(sklearn.metrics.mean_squared_error(predicted_ratings, actuals))
print "RMSE:", rmse

RMSE: 0.91227081992


<table>
    <tr>
        <th colspan=2><center>K=5</center></th>
    <tr>
        <th>$\lambda$</th>
        <th>RMSE</th>
    </tr>
    <tr>
        <td>0.001</td>
        <td>0.913146934761</td>
    </tr>
    <tr>
        <td>0.01</td>
        <td>0.912297969849</td>
    </tr>
    <tr>
        <td>0.1</td> 
        <td>0.913277052092</td>
    </tr>
    <tr>
        <td>1</td>
        <td>0.916509506566</td>
    </tr>
    <tr>
        <td>10</td>
        <td>1.01069785051</td>
    </tr>
</table>

<table>
    <tr>
        <th colspan=2><center>K=10</center></th>
    <tr>
        <th>$\lambda$</th>
        <th>RMSE</th>
    </tr>
    <tr>
        <td>0.0001</td>
        <td>0.91227081992</td>
    </tr>
    <tr>
        <td>0.001</td>
        <td>0.908526077502</td>
    </tr>
    <tr>
        <td>0.01</td>
        <td>0.910726843867</td>
    </tr>
    <tr>
        <td>0.1</td> 
        <td></td>
    </tr>
    <tr>
        <td>1</td>
        <td></td>
    </tr>
</table>

Next Steps
==========

1. Various methods have been devised to combine collaborative filtering and content-based recommendations into hybrid recommender systems (For example: https://www.microsoft.com/en-us/research/wp-content/uploads/2009/01/www09.pdf). 

2. Session-based click stream can be used instead of past user preferences, using RNNs (https://arxiv.org/pdf/1511.06939.pdf).

3. Long-tail recommendations are hard. Recommendation engines tend to recommend the most popular items. 

4. Use frameworks :) Faster and easier. http://surpriselib.com